In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["font.size"] = 12

In [2]:
import inspect

import numpy as np
import xarray as xr
import xarray_sentinel

from sarsen import apps, geocoding, orbit, scene

# uncomment to check that the code below is in sync with the implementation
# print(inspect.getsource(apps.backward_geocode_sentinel1))

# define input and load data

In [3]:
product_urlpath = (
    "data/S1B_IW_GRDH_1SDV_20211223T051122_20211223T051147_030148_039993_5371.SAFE/"
)
measurement_group = "IW/VV"
dem_urlpath = "data/Rome-10m-DEM.tif"
orbit_group = None
calibration_group = None
output_urlpath = "Rome-10m-GTC-GRD.tif"
correct_radiometry = False
interp_method = "nearest"
multilook = None
grouping_area_factor = (1.0, 1.0)
open_dem_raster_kwargs = {"chunks": {}}
kwargs = {"chunks": 2048}

In [4]:
!ls -d {product_urlpath}
!ls -d {dem_urlpath}

data/S1B_IW_GRDH_1SDV_20211223T051122_20211223T051147_030148_039993_5371.SAFE/
data/Rome-10m-DEM.tif


In [5]:
orbit_group = orbit_group or f"{measurement_group}/orbit"
calibration_group = calibration_group or f"{measurement_group}/calibration"

measurement_ds = xr.open_dataset(product_urlpath, engine="sentinel-1", group=measurement_group, **kwargs)  # type: ignore
measurement = measurement_ds.measurement

dem_raster = scene.open_dem_raster(dem_urlpath, **open_dem_raster_kwargs)

orbit_ecef = xr.open_dataset(product_urlpath, engine="sentinel-1", group=orbit_group, **kwargs)  # type: ignore
position_ecef = orbit_ecef.position
calibration = xr.open_dataset(product_urlpath, engine="sentinel-1", group=calibration_group, **kwargs)  # type: ignore
beta_nought_lut = calibration.betaNought

# scene

In [6]:
dem_raster

<xarray.DataArray 'band_data' (y: 5000, x: 5000)>
dask.array<getitem, shape=(5000, 5000), dtype=float32, chunksize=(5000, 5000), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 2.75e+05 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
  * y            (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06
    spatial_ref  int64 ...

In [7]:
# _ = dem_raster.plot()

In [8]:
%%time
dem_ecef = scene.convert_to_dem_ecef(dem_raster)
dem_ecef

CPU times: user 12.9 s, sys: 1.06 s, total: 14 s
Wall time: 13.6 s


<xarray.DataArray 'dem_3d' (axis: 3, y: 5000, x: 5000)>
dask.array<setitem, shape=(3, 5000, 5000), dtype=float64, chunksize=(3, 2500, 2500), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 2.75e+05 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
    spatial_ref  int64 ...
  * y            (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06
  * axis         (axis) int64 0 1 2

# acquisition

In [9]:
measurement

<xarray.DataArray 'measurement' (azimuth_time: 16705, ground_range: 26102)>
dask.array<open_dataset-f337086f3f5b0cc6653df3a7bbc965e4measurement, shape=(16705, 26102), dtype=float32, chunksize=(2048, 2048), chunktype=numpy.ndarray>
Coordinates:
    pixel         (ground_range) int64 dask.array<chunksize=(2048,), meta=np.ndarray>
    line          (azimuth_time) int64 dask.array<chunksize=(2048,), meta=np.ndarray>
  * azimuth_time  (azimuth_time) datetime64[ns] 2021-12-23T05:11:22.594441 .....
  * ground_range  (ground_range) float64 0.0 10.0 20.0 ... 2.61e+05 2.61e+05
Attributes:
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  10.0
    sar:pixel_spacing_range:    10.0
    azimuth_time_interval:      0.00149656999624572
    slant_range_time_interval:  1.554116558005821e-08
    sat:anx_datetime:           2021-12-23T04:33:35.556206Z
    units:                      1
    long_name:                  digital number

In [10]:
%%time
acquisition = apps.simulate_acquisition(position_ecef, dem_ecef)
acquisition

CPU times: user 51.1 s, sys: 18.2 s, total: 1min 9s
Wall time: 12.7 s


<xarray.Dataset>
Dimensions:           (x: 5000, y: 5000, axis: 3)
Coordinates:
  * x                 (x) float64 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
    spatial_ref       int64 0
  * y                 (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06
  * axis              (axis) int64 0 1 2
Data variables:
    azimuth_time      (y, x) datetime64[ns] dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    slant_range_time  (y, x) float64 dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    dem_direction     (axis, y, x) float64 dask.array<chunksize=(3, 2500, 2500), meta=np.ndarray>

In [11]:
%%time
beta_nought = xarray_sentinel.calibrate_intensity(measurement, beta_nought_lut)
beta_nought

CPU times: user 8.02 ms, sys: 747 µs, total: 8.76 ms
Wall time: 8.3 ms


<xarray.DataArray (azimuth_time: 16705, ground_range: 26102)>
dask.array<pow, shape=(16705, 26102), dtype=float32, chunksize=(2048, 2048), chunktype=numpy.ndarray>
Coordinates:
    pixel         (ground_range) int64 dask.array<chunksize=(2048,), meta=np.ndarray>
    line          (azimuth_time) int64 dask.array<chunksize=(2048,), meta=np.ndarray>
  * azimuth_time  (azimuth_time) datetime64[ns] 2021-12-23T05:11:22.594441 .....
  * ground_range  (ground_range) float64 0.0 10.0 20.0 ... 2.61e+05 2.61e+05
Attributes:
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  10.0
    sar:pixel_spacing_range:    10.0
    azimuth_time_interval:      0.00149656999624572
    slant_range_time_interval:  1.554116558005821e-08
    sat:anx_datetime:           2021-12-23T04:33:35.556206Z
    units:                      m2 m-2
    long_name:                  beta nought

In [12]:
coordinate_conversion = xr.open_dataset(
    product_urlpath,
    engine="sentinel-1",
    group=f"{measurement_group}/coordinate_conversion",
    **kwargs,
)  # type: ignore
ground_range = xarray_sentinel.slant_range_time_to_ground_range(
    acquisition.azimuth_time,
    acquisition.slant_range_time,
    coordinate_conversion,
)
interp_kwargs = {"ground_range": ground_range}
ground_range

<xarray.DataArray (y: 5000, x: 5000)>
dask.array<sum-aggregate, shape=(5000, 5000), dtype=float64, chunksize=(2500, 2500), chunktype=numpy.ndarray>
Coordinates:
    azimuth_time  (y, x) datetime64[ns] dask.array<chunksize=(2500, 2500), meta=np.ndarray>
  * x             (x) float64 2.75e+05 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
    spatial_ref   int64 0
  * y             (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06

In [13]:
%%time
geocoded = apps.interpolate_measurement(
    beta_nought,
    multilook=multilook,
    azimuth_time=acquisition.azimuth_time,
    interp_method=interp_method,
    **interp_kwargs,
)

geocoded

CPU times: user 16.8 s, sys: 8.28 s, total: 25.1 s
Wall time: 5.86 s


<xarray.DataArray (y: 5000, x: 5000)>
dask.array<dask_aware_interpnd, shape=(5000, 5000), dtype=float32, chunksize=(2500, 2500), chunktype=numpy.ndarray>
Coordinates:
    pixel         (y, x) float64 dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    line          (y, x) float64 dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    azimuth_time  (y, x) datetime64[ns] dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    ground_range  (y, x) float64 dask.array<chunksize=(2500, 2500), meta=np.ndarray>
  * x             (x) float64 2.75e+05 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
    spatial_ref   int64 0
  * y             (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06
Attributes:
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  10.0
    sar:pixel_spacing_range:    10.0
    azimuth_time_interval:      0.00149656999624572
    slant_range_time_interval:  1.554116558005821e-08
    sat:anx_datetime:           2021-12-23T04:33:35.556206Z
    units:                      m2 m-2
    long_name:                  beta nought

In [14]:
geocoded.rio.set_crs(dem_raster.rio.crs)
geocoded.rio.to_raster(
    output_urlpath,
    dtype=np.float32,
    tiled=True,
    blockxsize=512,
    blockysize=512,
    compress="ZSTD",
    num_threads="ALL_CPUS",
)

In [15]:
# _ = geocoded.plot(vmax=1.0)